In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-02 17:21:34--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  31.2MB/s    in 43s     

2021-01-02 17:22:17 (43.3 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-02 17:22:17--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [ ]:
!cp ../drive/MyDrive/slt/late-fusion/*.py signjoey/

In [ ]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [ ]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

## Keypoints

In [ ]:
body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:46<00:00, 152.73it/s]


## Hidden features

In [ ]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami2.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:33<00:00, 75.56it/s]


## Change config files

In [ ]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(3)]

list_doc['data']['feature_size'] = [1024, 417, 1024]
list_doc['model']['model_dir'] = "late_fusion"

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Run

In [ ]:
!cp -r '../drive/MyDrive/slt/models/baseline_20' baseline
!cp -r '../drive/MyDrive/slt/models/keypoints3d_20' keypoints3d
!cp -r '../drive/MyDrive/slt/models/hidden_features_20' features

cp: cannot stat '../drive/MyDrive/slt/models/keypoints_20': No such file or directory


In [ ]:
!cp -r '../drive/MyDrive/slt/models/keypoints3d_20' keypoints3d


In [ ]:
!ls baseline

2700.ckpt			     gls
best.ckpt			     gls.vocab
best.IT_00002700.BW_010.dev.gls      references.dev.gls
best.IT_00002700.BW_010.test.gls     references.dev.txt
best.IT_00002700.BW_04.A_2.dev.txt   tensorboard
best.IT_00002700.BW_04.A_2.test.txt  train.log
best.IT_00002700.dev_results.pkl     txt
best.IT_00002700.test_results.pkl    txt.vocab
config.yaml			     validations.txt


In [ ]:
%cd slt

/content/slt


In [ ]:
import os
import sys
from signjoey.prediction import test

test(cfg_file="configs/sign.yaml", ckpts=["baseline/best.ckpt", "keypoints3d/best.ckpt", "features/best.ckpt"], output_path="")

2021-01-02 17:45:48,711 - ------------------------------------------------------------
2021-01-02 17:45:48,716 - [DEV] partition [RECOGNITION] experiment [BW]: 1
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
2021-01-02 17:46:23,177 - finished in 34.4609s 
2021-01-02 17:46:23,179 - ************************************************************
2021-01-02 17:46:23,180 - [DEV] partition [RECOGNITION] results:
	New Best CTC Decode Beam Size: 1
	WER 63.73	(DEL: 61.78,	INS: 0.13,	SUB: 1.81)
2021-01-02 17:46:23,180 - ************************************************************
2021-01-02 17:46:23,181 - ------------------------------------------------------------
2021-01-02 17:46:23,182 - [DEV] partition [RECOGNITION] exper

translation


2021-01-02 18:03:41,823 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 1 and Alpha: -1
	BLEU-4 19.41	(BLEU-1: 45.26,	BLEU-2: 31.84,	BLEU-3: 24.21,	BLEU-4: 19.41)
	CHRF 42.41	ROUGE 45.93
2021-01-02 18:03:41,824 - ------------------------------------------------------------
2021-01-02 18:07:41,387 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: -1
	BLEU-4 19.65	(BLEU-1: 45.21,	BLEU-2: 31.98,	BLEU-3: 24.41,	BLEU-4: 19.65)
	CHRF 42.10	ROUGE 46.62
2021-01-02 18:07:41,389 - ------------------------------------------------------------
2021-01-02 18:08:50,731 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: 1
	BLEU-4 20.00	(BLEU-1: 45.73,	BLEU-2: 32.41,	BLEU-3: 24.79,	BLEU-4: 20.00)
	CHRF 42.48	ROUGE 46.77
2021-01-02 18:08:50,732 - ------------------------------------------------------------
2021-01-02 18:13:03,909 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 3 a

In [ ]:
58.47 & 20.35 & 57.38 & 20.11